$\bf{Problem\ Statement:}$ Write a Python function that takes the name of a star and returns a galpy Orbit object (setup with the star's position, distance, proper motion, and radial velocity). Apply to: GJ 440, Lacaille 8760, Vega. Check that you are finding the right star.

In [51]:
from galpy.potential import MWPotential2014
from galpy.orbit import Orbit
from gaia_tools import query
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
from numpy.ma import is_masked

Write a function that takes the name of star and return position and velocity from Simbad

In [52]:
def simbad_coord(name):
    """
    Take a star's name and return the position and velocity from Simbad.
    
    Args:
        name (string): the name of given star
        
    Returns:
        a list of 6 Quantity that represents the star's positioon and velocity:
        [ra, dec, parallax, pm_ra, pm_dec, vr]
        where
        ra (str): the right ascension in d:m:s
        dec (str): the declination in h:m:s
        parallx (float): the parallax in mas
        pm_ra (float): the proper motion in the right ascension direction in mas/yr
        pm_dec (float): the proper motion in the declination direction in mas/yr
        radial_velocity (float / None): the radial velocity in km/s; if not given by simbad, this is None
    """
    # create a Simbad object
    customSimbad = Simbad()
    # since the usual fields only include the 2 diensional coordinates, add parallax and velocity
    customSimbad.add_votable_fields('plx', 'pmra','pmdec', 'rv_value')
    # query simbad by the name
    simbad_table = customSimbad.query_object(name)
    result = [simbad_table['RA'][0], simbad_table['DEC'][0], simbad_table['PLX_VALUE'][0], 
              simbad_table['PMRA'][0], simbad_table['PMDEC'][0], simbad_table['RV_VALUE'][0]]
    # take care of the case when there is no raidal velocity
    if is_masked(result[5]):
        result[5] = None

    return result


In [53]:
simbad_coord('Lacaille 8760')

['21 17 15.2687', '-38 52 02.495', 253.41, -3260.29, -1146.22, 20.11]

Write the SQL query statement

In [2]:
#sql = "SELECT * FROM gaiadr2.gaia_source WHERE ra BETWEEN"
#out = query.query()

Create galpy orbit, given a star's position, distance, proper motion, and radial velocity

In [ ]:
def orbit_from_name(name):
    """
    Take a star's name and return a galpy Orbit object.
    
    Args:
        name (string): the name of given star
        
    Returns:
        o (Orbit): a galpy orbit object that represents the star's orbit
    """
    # get the position and velocity in ICRS form from Simbad; search by name
    ra, dec, parallax, pm_ra, pm_dec, vr = simbad_coord(name)
    # since these coordinates are from J2000, use Gaia catalogue to find its new coordinate in current epoch
    ra, dec = correct_for_current_epoch(ra, dec, parallax, pm_ra, pm_dec, vr = None)
    # query Gaia catalogue for the star's coordinate adn velocity; the gaia's data has a 'g' prefix
    g_ra, g_dec, g_parallax, g_pm_ra, g_pm_dec, g_vr = gaia_coord(ra, dec)
    
    # if gaia does not contain the star, use simbad's result; but otherwise, prefer gaia's result
    if g_ra is None:
        data = (ra, dec, parallax, pm_ra, pm_dec, vr)
    else:
        data = (g_ra, g_dec, g_parallax, g_pm_ra, g_pm_dec, g_vr)
    
    # take care of the case where no radial velocity is given in Gaia
    data = check_for_missing_vr(data, vr)
    # convert the coordinate to galactocentric
    R,vR,vT,z,vz,phi = convert_to_galactocentric(data)
    # create the galpy object and return the result
    o = Orbit(vxvv = [R,vR,vT,z,vz,phi])
    return o